# Geographic Optimization

## 1. Distance Calculation

Distances between each of the 16 focus compounds and all approx. 8000 zip codes in Germany.

### Imports

In [1]:
import pandas as pd
import numpy as np
import pgeocode
import haversine as hs

In [3]:
! pip install osrm-py

  Created wheel for osrm-py: filename=osrm_py-0.5-py3-none-any.whl size=5395 sha256=ef60444a69d69812c881fba6ed5c1c188de367e6737fb93791ab47db877154c3
  Stored in directory: /Users/lenasommer/Library/Caches/pip/wheels/d3/89/e3/6b52fbe15f2dd5ad6769c27e06ffb8670cd58da20c34518992
Successfully built osrm-py


In [4]:
import osrm

### Data Preparation

#### Create Dataframes

In [5]:
zipcodes_df = pd.read_csv('zipcodes.csv',usecols=['zipcode'],dtype='str')
zipcodes_df

,zipcode
0,01067
1,01069
2,01097
3,01099
4,01108
...,...
8169,99988
8170,99991
8171,99994
8172,99996


In [6]:
compounds_df = pd.read_csv('compounds_addresses.csv')
compounds_df

,compound_name,compound_address
0,AKB Kitzingen,"AKB Compound Kitzingen, Larson Barracks 53, 97..."
1,AKB Dortmund,"AKB Compound Dortmund, Dammstraße 25, 44145 Do..."
2,AKB Zörbig,"AKB Compound Zörbig, Jeßnitzer Str. 26, 06780 ..."
3,AKB Schöneck,"AKB Compound Schöneck, Windecker Str. 2, 61137..."
4,AKB Buch,"AKB Compound Buch, An der Lehmgrube 1, 89290 Buch"
5,Mosolf Etzin,ACM Auto-Service und Umschlag-Center Mosolf Et...
6,Mosolf Kippenheim,"Mosolf Compound, Freimatte 25, 77971 Kippenheim"
7,BLG Kelheim,"Hafenstraße 33, 93342 Saal an der Donau"
8,BLG Duisburg,"BLG AutoTerminal Duisburg GmbH & Co. KG, Rotte..."
9,BLG Neuss,"ATN Autoterminal Neuss, Floßhafenstr. 30, 4146..."


#### Convert zip code to longitude and latitude

In [7]:
nomi = pgeocode.Nominatim('de')

In [8]:
nomi.query_postal_code(zipcodes_df.iat[0,0])

postal_code                                                   01067
country_code                                                     DE
place_name        Dresden Innere Altstadt, Dresden, Dresden Frie...
state_name                                                  Sachsen
state_code                                                       SN
county_name                                                     NaN
county_code                                                     0.0
community_name                             Kreisfreie Stadt Dresden
community_code                                              14612.0
latitude                                                    51.0547
longitude                                                   13.7269
accuracy                                                        4.0
Name: 0, dtype: object

In [9]:
for index,row in zipcodes_df.iterrows():
    query = nomi.query_postal_code(zipcodes_df.iat[index,0])
    zipcodes_df.at[index,'lat']= query['latitude']
    zipcodes_df.at[index,'long']= query['longitude']
    zipcodes_df.at[index,'state']= query['state_name']
    zipcodes_df.at[index,'state_code']= query['state_code']
    zipcodes_df.at[index,'community_name']= query['community_name']
    zipcodes_df.at[index,'community_code']= query['community_code']

In [10]:
compounds_df['zipcode'] = compounds_df['compound_address'].str.findall(r'([0-9]\d+)').apply(lambda x: x[-1] if len(x) >= 1 else '')


In [11]:
for index,row in compounds_df.iterrows():
    query = nomi.query_postal_code(compounds_df.iat[index,2])
    compounds_df.at[index,'lat']= query['latitude']
    compounds_df.at[index,'long']= query['longitude']

In [12]:
compounds_df

,compound_name,compound_address,zipcode,lat,long
0,AKB Kitzingen,"AKB Compound Kitzingen, Larson Barracks 53, 97...",97318,49.75320,10.11700
1,AKB Dortmund,"AKB Compound Dortmund, Dammstraße 25, 44145 Do...",44145,51.52900,7.48020
2,AKB Zörbig,"AKB Compound Zörbig, Jeßnitzer Str. 26, 06780 ...",06780,51.61855,12.13373
3,AKB Schöneck,"AKB Compound Schöneck, Windecker Str. 2, 61137...",61137,50.20560,8.83330
4,AKB Buch,"AKB Compound Buch, An der Lehmgrube 1, 89290 Buch",89290,48.22480,10.18050
5,Mosolf Etzin,ACM Auto-Service und Umschlag-Center Mosolf Et...,14669,52.48090,12.81490
6,Mosolf Kippenheim,"Mosolf Compound, Freimatte 25, 77971 Kippenheim",77971,48.29560,7.82510
7,BLG Kelheim,"Hafenstraße 33, 93342 Saal an der Donau",93342,48.90000,11.93330
8,BLG Duisburg,"BLG AutoTerminal Duisburg GmbH & Co. KG, Rotte...",47229,51.39100,6.70710
9,BLG Neuss,"ATN Autoterminal Neuss, Floßhafenstr. 30, 4146...",41460,51.20140,6.69440


#### Add coordinate column (necessary for usage of Haversine) 

In [58]:
zipcodes_df['coor']=list(zip(zipcodes_df.lat,zipcodes_df.long))
compounds_df['coor']=list(zip(compounds_df.lat,compounds_df.long))

In [14]:
zipcodes_df

,zipcode,lat,long,state,state_code,community_name,community_code,coor
0,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)"
1,01069,51.043000,13.737300,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.043, 13.7373)"
2,01097,51.071400,13.739900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0714, 13.7399)"
3,01099,51.078300,13.805100,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0783, 13.8051)"
4,01108,51.155733,13.782467,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.15573333333333, 13.782466666666666)"
...,...,...,...,...,...,...,...,...
8169,99988,51.172900,10.290450,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.1729, 10.29045)"
8170,99991,51.148467,10.553300,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.14846666666667, 10.5533)"
8171,99994,51.239850,10.670850,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.23985, 10.67085)"
8172,99996,51.288800,10.580350,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.2888, 10.58035)"


### Calculate Distances

In [20]:
def distance_from(loc1,loc2):
    '''This function defines the distance between customers (loc1) and compound (loc2)'''
    dist = hs.haversine(loc1,loc2)
    return round(dist,2)

In [21]:
full_distances_df = zipcodes_df.copy()

In [22]:
for _,row in compounds_df.iterrows():
    full_distances_df[row.compound_name]=full_distances_df['coor'].apply(lambda x: distance_from(row['coor'],x))

In [23]:
distances = full_distances_df.drop(columns=['lat','long','coor'],axis=1)

In [24]:
distances.set_index('zipcode', inplace=True)

In [25]:
distances

,state,state_code,community_name,community_code,AKB Kitzingen,AKB Dortmund,AKB Zörbig,AKB Schöneck,AKB Buch,Mosolf Etzin,Mosolf Kippenheim,BLG Kelheim,BLG Duisburg,BLG Neuss,Carservice Erkens,ARS Altmann Wolnzach,CAT Zülpich,Autohaus Siebrecht,Neu: FleetParQ Essen,Neu: FleetParQ Kassel
zipcode,,,,,,,,,,,,,,,,,,,,
01067,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,293.89,437.43,127.20,357.77,405.16,170.55,523.64,271.74,490.10,490.84,518.92,311.40,497.90,291.88,465.29,293.80
01069,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,293.92,438.36,128.48,358.18,404.64,172.02,523.53,270.96,490.99,491.67,519.80,310.63,498.59,292.92,466.20,294.69
01097,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,295.57,438.02,127.07,359.08,407.16,169.15,525.43,273.81,490.78,491.60,519.59,313.46,498.87,292.29,465.92,294.49
01099,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,299.95,442.40,130.70,363.69,410.71,170.19,529.65,276.70,495.21,496.08,524.02,316.41,503.45,296.51,470.32,298.93
01108,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,302.71,439.58,125.45,364.28,416.25,161.66,533.13,283.46,492.70,493.93,521.50,323.08,502.25,292.98,467.63,296.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99988,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,158.34,199.13,137.14,148.67,327.91,226.36,365.68,278.63,250.38,250.62,279.17,301.25,260.43,70.45,226.25,54.78
99991,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,158.19,217.61,121.49,160.24,326.20,214.74,372.89,268.74,268.90,269.05,297.69,293.20,278.06,85.17,244.77,73.22
99994,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,169.89,223.72,109.81,173.07,337.10,201.78,385.88,275.33,275.96,276.94,304.78,300.97,287.84,85.46,251.32,79.66


Result:
For every zipcode, the distances (in km) to every compound are given. 
As it's stored in a pandas Dataframe, further investigations can be easily done (p.eg. seeing the minimum per row etc.).

### Calculate Driving Distance

In [18]:
import osrm
import requests
import json

In [35]:
zipcodes_df.coor[0]

(51.0547, 13.7269)

In [55]:
coor_plz = zipcodes_df.coor
coor_plz

0                            (51.0547, 13.7269)
1                             (51.043, 13.7373)
2                            (51.0714, 13.7399)
3                            (51.0783, 13.8051)
4       (51.15573333333333, 13.782466666666666)
                         ...                   
8169                        (51.1729, 10.29045)
8170               (51.14846666666667, 10.5533)
8171                       (51.23985, 10.67085)
8172                        (51.2888, 10.58035)
8173                         (51.2125, 10.5625)
Name: coor, Length: 8174, dtype: object

In [119]:
all_compounds_index=";".join([f"{i}" for i in range(len(compounds_df))])
all_zip_index=";".join([f"{i+16}" for i in range(len(zipcodes_df))])

prefix = "http://router.project-osrm.org/table/v1/driving/"
sources = f"?sources={all_compounds_index}"
destinations = f"&destinations={all_zip_index}"
distance = "?annotations=distance"

body_zips = ";".join([f"{zipcodes_df.long[i]},{zipcodes_df.lat[i]}" for i in range(len(zipcodes_df))])
body_compounds = ";".join([f"{compounds_df.long[j]},{compounds_df.lat[j]}" for j in range(len(compounds_df))])

r = requests.get(prefix+body_compounds+";"+body_zips+sources+destinations+distance)
r.raise_for_status()
content = json.loads(r.content)
content

HTTPError: 414 Client Error: Request-URI Too Large for url: http://router.project-osrm.org/table/v1/driving/10.117,49.7532;7.4802,51.529;12.13373,51.618550000000006;8.8333,50.2056;10.1805,48.2248;12.8149,52.4809;7.8251,48.2956;11.9333,48.9;6.7071,51.391;6.6944,51.2014;6.2919,51.3974;11.6257,48.6038;6.65,50.7;9.635,51.6569;7.0714,51.4804;9.5297,51.2988;13.7269,51.0547;13.7373,51.043;13.7399,51.0714;13.8051,51.0783;13.782466666666666,51.15573333333333;13.7468,51.1164;13.7208,51.079;13.736,51.0967;13.6981,51.0836;13.62143,51.04942;13.6729,51.0707;13.6996,51.0452;13.6738,51.0443;13.6943,51.0281;13.75,51.05;13.75,51.05;13.7679,51.021;13.7988,51.013;13.8089,51.0066;13.8054,51.0114;13.75,51.05;13.787033333333332,51.042566666666666;13.7801,51.0425;13.75,51.05;13.77835,51.05095;13.75,51.05;13.8155,51.0538;13.75,51.05;13.6605,51.1065;13.90925,51.1364;13.8333,51.1833;13.8,51.09165;13.6802,51.1592;13.7281,51.2152;13.9918,51.0942;13.5335,51.2895;13.635777777777776,51.2955;13.2848,51.2991;13.3097,51

In [114]:
";".join([f"{i+16}" for i in range(len(zipcodes_df))])


'16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;69;70;71;72;73;74;75;76;77;78;79;80;81;82;83;84;85;86;87;88;89;90;91;92;93;94;95;96;97;98;99;100;101;102;103;104;105;106;107;108;109;110;111;112;113;114;115;116;117;118;119;120;121;122;123;124;125;126;127;128;129;130;131;132;133;134;135;136;137;138;139;140;141;142;143;144;145;146;147;148;149;150;151;152;153;154;155;156;157;158;159;160;161;162;163;164;165;166;167;168;169;170;171;172;173;174;175;176;177;178;179;180;181;182;183;184;185;186;187;188;189;190;191;192;193;194;195;196;197;198;199;200;201;202;203;204;205;206;207;208;209;210;211;212;213;214;215;216;217;218;219;220;221;222;223;224;225;226;227;228;229;230;231;232;233;234;235;236;237;238;239;240;241;242;243;244;245;246;247;248;249;250;251;252;253;254;255;256;257;258;259;260;261;262;263;264;265;266;267;268;269;270;271;272;273;274;275;276;277;278;279;280;281;282;283;284;285;286

In [75]:
zipcodes_df

,zipcode,lat,long,state,state_code,community_name,community_code,coor
0,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)"
1,01069,51.043000,13.737300,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.043, 13.7373)"
2,01097,51.071400,13.739900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0714, 13.7399)"
3,01099,51.078300,13.805100,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0783, 13.8051)"
4,01108,51.155733,13.782467,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.15573333333333, 13.782466666666666)"
...,...,...,...,...,...,...,...,...
8169,99988,51.172900,10.290450,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.1729, 10.29045)"
8170,99991,51.148467,10.553300,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.14846666666667, 10.5533)"
8171,99994,51.239850,10.670850,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.23985, 10.67085)"
8172,99996,51.288800,10.580350,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.2888, 10.58035)"


In [86]:
r = requests.get(f"""http://router.project-osrm.org/table/v1/driving/{(zipcodes_df.long[0])},{(zipcodes_df.lat[0])};{(compounds_df.long[0])},{(compounds_df.lat[0])};{(compounds_df.long[1])},{(compounds_df.lat[1])}?annotations=distance""")
content = json.loads(r.content)
content
content['distances'][0][1]

374012.8

In [26]:
import requests
import json
from tqdm import tqdm

In [27]:
def request_driving_distance_in_meters_from_api(loc1,loc2):
    '''Requests from OpenStreetMap to calculate Driving Distance between customer and compound'''
    r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{loc1[1]},{loc1[0]};{loc2[1]},{loc2[0]}?overview=false""")
    content = json.loads(r.content)
    if 'routes' in content:
        route_1 = content['routes'][0]
        return route_1['distance']
    else:
        return 0.0

In [28]:
tqdm.pandas()
driving_distances_df = zipcodes_df.head(100)
for _,row in compounds_df.iterrows():
    driving_distances_df[row.compound_name]=driving_distances_df['coor'].progress_apply(lambda x: request_driving_distance_in_meters_from_api(row['coor'],x))

100%|██████████| 100/100 [00:50<00:00,  1.96it/s]
/var/folders/0y/smycr6594jxd0n8scvb22wph0000gn/T/ipykernel_47425/4044339078.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driving_distances_df[row.compound_name]=driving_distances_df['coor'].progress_apply(lambda x: request_driving_distance_in_meters_from_api(row['coor'],x))
100%|██████████| 100/100 [00:49<00:00,  2.00it/s]


In [29]:
driving_distances_df

,zipcode,lat,long,state,state_code,community_name,community_code,coor,AKB Kitzingen,AKB Dortmund,...,Mosolf Kippenheim,BLG Kelheim,BLG Duisburg,BLG Neuss,Carservice Erkens,ARS Altmann Wolnzach,CAT Zülpich,Autohaus Siebrecht,Neu: FleetParQ Essen,Neu: FleetParQ Kassel
0,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",373895.3,516386.3,...,643658.0,359656.3,595467.3,584339.3,622441.4,405654.4,617219.7,352558.6,553038.0,350319.5
1,01069,51.043000,13.737300,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.043, 13.7373)",380735.6,523226.5,...,650498.2,366496.6,602307.6,591179.5,629281.6,412494.6,624060.0,359398.9,559878.3,357159.7
2,01097,51.071400,13.739900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0714, 13.7399)",375136.0,517627.0,...,644898.7,360897.0,596708.0,585580.0,623682.1,406895.1,618460.4,353799.3,554278.7,351560.2
3,01099,51.078300,13.805100,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0783, 13.8051)",379768.4,522259.4,...,649531.1,365529.4,601340.4,590212.4,628314.4,411527.5,623092.8,358431.7,558911.1,356192.6
4,01108,51.155733,13.782467,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.15573333333333, 13.782466666666666)",388765.1,531256.0,...,658527.7,374526.1,610337.1,599209.1,637311.1,420524.1,632089.5,367428.4,567907.8,365189.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,01996,51.454700,14.025100,Brandenburg,BB,Landkreis Oberspreewald-Lausitz,12066.0,"(51.4547, 14.0251)",433056.9,575547.8,...,702819.5,418817.9,654628.9,643500.8,681602.9,464815.9,676381.3,411720.2,612199.6,409481.0
96,01998,51.546067,13.911967,Brandenburg,BB,Landkreis Oberspreewald-Lausitz,12066.0,"(51.54606666666667, 13.911966666666666)",431242.5,573733.5,...,701005.2,417003.5,652814.5,641686.5,679788.6,463001.6,674566.9,409905.8,610385.2,407666.7
97,02625,51.182500,14.429200,Sachsen,SN,Landkreis Bautzen,14625.0,"(51.1825, 14.4292)",430707.7,573198.6,...,700470.3,416468.6,652279.7,641151.6,679253.7,462466.7,674032.1,409370.9,609850.3,407131.8
98,02627,51.189225,14.531925,Sachsen,SN,Landkreis Bautzen,14625.0,"(51.189225, 14.531925)",439252.3,581743.3,...,709015.0,425013.3,660824.3,649696.3,687798.4,471011.4,682576.7,417915.6,618395.0,415676.5


## 3. Heatmap

Visualize which cars are demanded by which customers in which regions of Germany.

In [30]:
analysis_input_df = pd.read_csv('zip_code_analysis_input.csv',usecols=['sub_property_handover_zipcode','sub_property_handover_city','abs_net_purchase_price','brand_name','config_model_name','finn_car_id','deal_id','dim_fkey_pipelinestage','product_brand','purchasing_model','purchasing_model_line','product_fuel','helper_subscription_handover_date','product_body_type','delivery_compound_location'],dtype='str')
analysis_input_df=analysis_input_df.rename(columns={'sub_property_handover_zipcode':'zipcode'})

In [31]:
full_geodata_df = pd.merge(zipcodes_df,analysis_input_df,how='inner',on='zipcode')
full_geodata_df

,zipcode,lat,long,state,state_code,community_name,community_code,coor,sub_property_handover_city,abs_net_purchase_price,...,finn_car_id,deal_id,dim_fkey_pipelinestage,product_brand,purchasing_model,purchasing_model_line,product_fuel,helper_subscription_handover_date,product_body_type,delivery_compound_location
0,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",Dresden,NaN,...,etw8hl87,6746684805,1319394,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
1,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",Dresden,39451.41,...,awmj43ax,10031089330,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",Dresden,23138.83,...,6a4y4psd,8264850799,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01069,51.043000,13.737300,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.043, 13.7373)","Dresden, Sachsen",NaN,...,w2riqyps,5364503057,1319394,Fiat,NaN,NaN,NaN,NaN,NaN,NaN
4,01069,51.043000,13.737300,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.043, 13.7373)",Dresden,28588.52,...,jsuh5jig,9185094506,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13882,99869,50.967996,10.706304,Thüringen,TH,Landkreis Gotha,16067.0,"(50.96799642857143, 10.706303571428572)",Drei Gleichen OT Wechmar,NaN,...,p30d722d,10720063262,1050363,Opel,NaN,NaN,NaN,NaN,NaN,NaN
13883,99880,50.930420,10.585770,Thüringen,TH,Landkreis Gotha,16067.0,"(50.93042, 10.58577)",Waltershausen,NaN,...,g23xc8qu,4663230905,1319394,Jeep,NaN,NaN,NaN,NaN,NaN,NaN
13884,99885,50.806800,10.748833,Thüringen,TH,Landkreis Gotha,16067.0,"(50.8068, 10.748833333333332)",Ohrdruf,NaN,...,g1durj3r,6297457305,1319394,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13885,99976,51.247783,10.328933,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.24778333333333, 10.328933333333334)",Rodeberg,15061.95,...,2atzum4f,8916763229,1050363,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
airtable_df = pd.read_csv('airtable_data.csv',dtype='str')
airtable_df

,finn_car_id,product_brand,purchasing_model,purchasing_model_line,product_fuel,helper_subscription_handover_date,product_body_type,delivery_compound_location
0,0a0jlxzj,Opel,Crossland,Edition,Benzin,NaN,SUV,compound_blg_saalanderdonau
1,0a4q0kww,Renault,Koleos,Intens,Benzin,2022-10-24,SUV,compound_rrg_eching
2,0a5aqavf,Jeep,Compass,Upland,Plug-In-Hybrid,NaN,SUV,compound_mosolf_kippenheim
3,0a5o8f6t,Audi,Q3,S line,Benzin (mild-hybrid),NaN,SUV,compound_akb_kitzingen
4,0a6jwfvb,Nissan,Qashqai,Acenta,Benzin (mild-hybrid),2022-06-02,SUV,compound_blg_d2c_atn2_neuss
...,...,...,...,...,...,...,...,...
42181,zzx60fa5,BMW,3er Touring,M Automobile,Diesel,2022-11-21,Kombi,compound_akb_kitzingen
42182,zzyasour,Jeep,Compass,Limited,Benzin,2022-06-28,SUV,compound_mosolf_kippenheim
42183,zzyg5ckg,Opel,Grandland,GS Line,Benzin,NaN,SUV,dealer_siebrecht_d2c_uslar
42184,zzz1n8rw,VW,Caravelle T6 1,Comfortline LR,Diesel,2023-02-09,Van,compound_akb_kitzingen


In [33]:
full_joined_df = pd.merge(full_geodata_df,airtable_df,how='inner',on='finn_car_id')
full_joined_df

,zipcode,lat,long,state,state_code,community_name,community_code,coor,sub_property_handover_city,abs_net_purchase_price,...,helper_subscription_handover_date_x,product_body_type_x,delivery_compound_location_x,product_brand_y,purchasing_model_y,purchasing_model_line_y,product_fuel_y,helper_subscription_handover_date_y,product_body_type_y,delivery_compound_location_y
0,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",Dresden,NaN,...,NaN,NaN,NaN,Fiat,500 e,ICON,Elektro,2022-07-06,Kleinwagen,compound_mosolf_kippenheim
1,97892,49.768200,9.518200,Bayern,BY,Main-Spessart,9677.0,"(49.7682, 9.5182)",Kreuzwertheim,NaN,...,NaN,NaN,NaN,Fiat,500 e,ICON,Elektro,2022-07-06,Kleinwagen,compound_mosolf_kippenheim
2,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",Dresden,39451.41,...,NaN,NaN,NaN,BMW,3er Limousine,M Sportpaket,Benzin,2022-10-04,Limousine,compound_akb_kitzingen
3,01067,51.054700,13.726900,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.0547, 13.7269)",Dresden,23138.83,...,NaN,NaN,NaN,Nissan,Qashqai,Acenta,Benzin (mild-hybrid),2022-03-29,SUV,compound_blg_d2c_atn2_neuss
4,01069,51.043000,13.737300,Sachsen,SN,Kreisfreie Stadt Dresden,14612.0,"(51.043, 13.7373)","Dresden, Sachsen",NaN,...,NaN,NaN,NaN,Fiat,500 e,ICON,Elektro,2022-10-18,Kleinwagen,compound_mosolf_kippenheim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13881,99834,50.968875,10.142300,Thüringen,TH,Wartburgkreis,16063.0,"(50.968875, 10.1423)",Gerstungen,0,...,NaN,NaN,NaN,VW,Passat Variant,Business,Benzin,2023-01-17,Kombi,compound_arsaltmann_wolnzach
13882,99867,50.948200,10.701900,Thüringen,TH,Landkreis Gotha,16067.0,"(50.9482, 10.7019)",Gotha,NaN,...,NaN,NaN,NaN,Jeep,Compass,Limited,Benzin,2022-05-09,SUV,compound_mosolf_kippenheim
13883,99867,50.948200,10.701900,Thüringen,TH,Landkreis Gotha,16067.0,"(50.9482, 10.7019)",Gotha,NaN,...,NaN,NaN,NaN,Jeep,Compass,Limited,Benzin,2022-04-20,SUV,compound_mosolf_kippenheim
13884,99976,51.247783,10.328933,Thüringen,TH,Unstrut-Hainich-Kreis,16064.0,"(51.24778333333333, 10.328933333333334)",Rodeberg,15061.95,...,NaN,NaN,NaN,Opel,Crossland,GS Line,Benzin,2022-07-06,SUV,dealer_siebrecht_d2c_uslar


In [34]:
from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

In [35]:
full_joined_df['lat']=full_joined_df['lat'].astype('float')
full_joined_df['long']=full_joined_df['long'].astype('float')

In [36]:
#Bokeh maps are in mercator. Convert lat lon fields to mercator units for plotting

def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df=wgs84_to_web_mercator(full_joined_df,'long','lat')

In [43]:
#Establishing a zoom scale for the map. The scale variable will also determine proportions for hexbins and bubble maps so that everything looks visually appealing. 

scale=2000
x=df['x']
y=df['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.

x_min=int(x.mean() - (scale * 350))
x_max=int(x.mean() + (scale * 350))
y_min=int(y.mean() - (scale * 350))
y_max=int(y.mean() + (scale * 350))

#Defining the map tiles to use. I use OSM, but you can also use ESRI images or google street maps.


#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='Location of Customers',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator'
    )

plot.grid.visible=True

map=plot.add_tile('OSM',retina=True)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False

#If in Jupyter, use the output_notebook() method to display the plot in-line. If not, you can use output_file() or another method to save your map. 
output_notebook()

Loading BokehJS ...

In [44]:
#function takes scale (defined above), the initialized plot object, and the converted dataframe with mercator coordinates to create a hexbin map

def hex_map(plot,df, scale,leg_label='Hexbin Heatmap'):
  r,bins=plot.hexbin(x,y,size=scale*10,hover_color='pink',hover_alpha=0.8,legend_label=leg_label)
  hex_hover = HoverTool(tooltips=[('count','@c')],mode='mouse',point_policy='follow_mouse',renderers=[r])
  hex_hover.renderers.append(r)
  plot.tools.append(hex_hover)
  
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

#function takes a column to determine radius and the dataframe with converted mercator coordinates to create a bubble map. 
def bubble_map(plot,df,radius_col,long,lat,scale,color='orange',leg_label='Bubble Map'):
  radius=[]
  for i in df[radius_col]:
    radius.append(i*scale)
  
  df['radius']=radius
    
  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius='radius',legend_label=leg_label,hover_color='red')

  tip_label='@'+radius_col
  lat_label='@'+lat
  lon_label='@'+long

  circle_hover = HoverTool(tooltips=[(radius_col,tip_label),('Lat:',lat_label),('Long:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

In [45]:
count_per_zipcode = full_joined_df[['finn_car_id','community_name']].groupby('community_name').count()
count_per_zipcode = count_per_zipcode.reset_index()

full_df = pd.merge(count_per_zipcode,full_joined_df,how='inner',on='community_name')
full_df = full_df.rename(columns={'finn_car_id_x':'count_customers'})

In [46]:
#Create the hexbin map
hex_map(plot=plot,
        df=full_df, 
        scale=scale,
        leg_label='Location of Customers')

In [41]:
#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.
#bubble_map(plot=plot,
#           df=full_df,
#           radius_col='count_customers', 
#           leg_label='Amount of Customers per Zipcode',
#           long='long',
#           lat='lat',
#           scale=scale)

In [47]:
show(plot)

In [52]:
output_notebook()
save(plot)

Loading BokehJS ...

'/Users/lenasommer/Documents/Arbeit/Finn /geographic_optimization/plot.html'

In [53]:
show(plot)